<a href="https://colab.research.google.com/github/Ramanaa-Vijayakumar/Smart-Product-Labeling-and-Traceability-System/blob/main/Fresh_vs_Rotton_Fruits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU Available: []


In [ ]:
# Install Kaggle API
!pip install kaggle

# Upload your kaggle.json file
from google.colab import files
import json

print("Please upload your kaggle.json file:")
print("1. Go to https://www.kaggle.com/account")
print("2. Click 'Create New API Token' to download kaggle.json")
print("3. Upload the file when prompted below")

uploaded = files.upload()

# Setup Kaggle credentials
for filename in uploaded.keys():
    if filename == 'kaggle.json':
        with open('/content/kaggle.json', 'w') as f:
            f.write(uploaded[filename].decode('utf-8'))
        break

# Set permissions and move to correct location
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle credentials setup complete!")

Please upload your kaggle.json file:
1. Go to https://www.kaggle.com/account
2. Click 'Create New API Token' to download kaggle.json
3. Upload the file when prompted below


KeyboardInterrupt: 

In [ ]:
# Manual dataset specification (if you know the exact Kaggle dataset)
# Uncomment and modify the line below if you want to specify a different dataset:
DATASET_NAME = "sriramr/fruits-fresh-and-rotten-for-classification"

print(f"Downloading dataset: {DATASET_NAME}")
!kaggle datasets download -d {DATASET_NAME}

# Extract the dataset
import zipfile
dataset_zip = f"{DATASET_NAME.split('/')[-1]}.zip"
print(f"Extracting {dataset_zip}...")

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/fruit_dataset')

# List contents to understand structure
print("Dataset structure:")
!find /content/fruit_dataset -type d | head -20

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
Extracting fruits-fresh-and-rotten-for-classification.zip...


FileNotFoundError: [Errno 2] No such file or directory: 'fruits-fresh-and-rotten-for-classification.zip'

In [ ]:
import glob

# Find the actual dataset structure
BASE_PATH = '/content/fruit_dataset'

# Search for train and test directories
train_candidates = glob.glob(f"{BASE_PATH}/**/train*", recursive=True)
test_candidates = glob.glob(f"{BASE_PATH}/**/test*", recursive=True)

print("Potential train directories:", train_candidates)
print("Potential test directories:", test_candidates)

# Auto-detect paths or set manually based on dataset structure
if train_candidates and test_candidates:
    TRAIN_PATH = train_candidates[0]
    TEST_PATH = test_candidates[0]
else:
    # If no explicit train/test folders, look for the main dataset folder
    dataset_folders = [f for f in os.listdir(BASE_PATH) if os.path.isdir(os.path.join(BASE_PATH, f))]
    print("Available folders:", dataset_folders)

    # Common patterns for fruit datasets
    possible_paths = [
        '/content/fruit_dataset/dataset',
        '/content/fruit_dataset/fruits',
        '/content/fruit_dataset'
    ]

    # Try to find the correct structure
    for path in possible_paths:
        if os.path.exists(path):
            subfolders = os.listdir(path)
            if 'train' in subfolders and 'test' in subfolders:
                TRAIN_PATH = os.path.join(path, 'train')
                TEST_PATH = os.path.join(path, 'test')
                break
            elif any('fresh' in folder.lower() or 'rotten' in folder.lower() for folder in subfolders):
                # If it's a flat structure, we'll organize it later
                TRAIN_PATH = path
                TEST_PATH = path
                break

print(f"\nSelected paths:")
print(f"Train path: {TRAIN_PATH}")
print(f"Test path: {TEST_PATH}")

# Verify paths exist and show contents
if os.path.exists(TRAIN_PATH):
    print(f"Train folders: {os.listdir(TRAIN_PATH)}")
if os.path.exists(TEST_PATH):
    print(f"Test folders: {os.listdir(TEST_PATH)}")

# If dataset doesn't have train/test split, we'll create one
CREATE_SPLIT = False
if TRAIN_PATH == TEST_PATH:
    print("\nDataset appears to be in a single folder. We'll create train/test split.")
    CREATE_SPLIT = True

Potential train directories: []
Potential test directories: []


FileNotFoundError: [Errno 2] No such file or directory: '/content/fruit_dataset'

In [ ]:
import shutil
from sklearn.model_selection import train_test_split

def create_train_test_split(source_path, train_ratio=0.8):
    """
    Create train/test split if dataset is in single folder
    """
    # Create directories
    split_train_path = os.path.join(BASE_PATH, 'split_train')
    split_test_path = os.path.join(BASE_PATH, 'split_test')

    for path in [split_train_path, split_test_path]:
        os.makedirs(path, exist_ok=True)

    # Get all folders in source path
    folders = [f for f in os.listdir(source_path) if os.path.isdir(os.path.join(source_path, f))]

    for folder in folders:
        folder_path = os.path.join(source_path, folder)

        # Create folder in both train and test
        os.makedirs(os.path.join(split_train_path, folder), exist_ok=True)
        os.makedirs(os.path.join(split_test_path, folder), exist_ok=True)

        # Get all images in folder
        images = [f for f in os.listdir(folder_path)
                 if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Split images
        train_images, test_images = train_test_split(
            images, train_size=train_ratio, random_state=42
        )

        # Copy to train folder
        for img in train_images:
            src = os.path.join(folder_path, img)
            dst = os.path.join(split_train_path, folder, img)
            shutil.copy2(src, dst)

        # Copy to test folder
        for img in test_images:
            src = os.path.join(folder_path, img)
            dst = os.path.join(split_test_path, folder, img)
            shutil.copy2(src, dst)

        print(f"Split {folder}: {len(train_images)} train, {len(test_images)} test")

    return split_train_path, split_test_path

def reorganize_data(base_path, train_path, test_path):
    """
    Reorganize data from fruit-specific folders to fresh/rotten binary classification
    """
    # Create new directory structure
    new_train_path = os.path.join(base_path, 'binary_train')
    new_test_path = os.path.join(base_path, 'binary_test')

    # Create directories
    for path in [new_train_path, new_test_path]:
        os.makedirs(os.path.join(path, 'fresh'), exist_ok=True)
        os.makedirs(os.path.join(path, 'rotten'), exist_ok=True)

    # Function to copy files
    def copy_files(source_path, dest_path):
        if os.path.exists(source_path):
            folders = os.listdir(source_path)

            for folder in folders:
                folder_path = os.path.join(source_path, folder)
                if os.path.isdir(folder_path):
                    # Determine if fresh or rotten
                    if 'fresh' in folder.lower():
                        target_dir = os.path.join(dest_path, 'fresh')
                    elif 'rotten' in folder.lower():
                        target_dir = os.path.join(dest_path, 'rotten')
                    else:
                        continue

                    # Copy all images from this folder
                    for img_file in os.listdir(folder_path):
                        if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                            src_file = os.path.join(folder_path, img_file)
                            # Create unique filename to avoid conflicts
                            dest_file = os.path.join(target_dir, f"{folder}_{img_file}")
                            if not os.path.exists(dest_file):
                                shutil.copy2(src_file, dest_file)

    # Copy training data
    copy_files(train_path, new_train_path)

    # Copy test data
    copy_files(test_path, new_test_path)

    return new_train_path, new_test_path

# Handle data organization
if CREATE_SPLIT:
    print("Creating train/test split...")
    TRAIN_PATH, TEST_PATH = create_train_test_split(TRAIN_PATH)

# Reorganize data for binary classification
print("Reorganizing data for binary classification...")
BINARY_TRAIN_PATH, BINARY_TEST_PATH = reorganize_data(BASE_PATH, TRAIN_PATH, TEST_PATH)

# Count images in each category
def count_images(path):
    counts = {}
    for category in ['fresh', 'rotten']:
        category_path = os.path.join(path, category)
        if os.path.exists(category_path):
            counts[category] = len([f for f in os.listdir(category_path)
                                 if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    return counts

train_counts = count_images(BINARY_TRAIN_PATH)
test_counts = count_images(BINARY_TEST_PATH)

print("\nDataset Summary:")
print(f"Training data: {train_counts}")
print(f"Test data: {test_counts}")
print(f"Total training images: {sum(train_counts.values())}")
print(f"Total test images: {sum(test_counts.values())}")

NameError: name 'CREATE_SPLIT' is not defined

In [ ]:
# Visualize data distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Training data distribution
ax1.bar(train_counts.keys(), train_counts.values(), color=['green', 'red'], alpha=0.7)
ax1.set_title('Training Data Distribution')
ax1.set_ylabel('Number of Images')
for i, v in enumerate(train_counts.values()):
    ax1.text(i, v + 50, str(v), ha='center', va='bottom')

# Test data distribution
ax2.bar(test_counts.keys(), test_counts.values(), color=['green', 'red'], alpha=0.7)
ax2.set_title('Test Data Distribution')
ax2.set_ylabel('Number of Images')
for i, v in enumerate(test_counts.values()):
    ax2.text(i, v + 50, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Display sample images
def display_sample_images(path, num_samples=3):
    fig, axes = plt.subplots(2, num_samples, figsize=(15, 8))

    for i, category in enumerate(['fresh', 'rotten']):
        category_path = os.path.join(path, category)
        if os.path.exists(category_path):
            images = [f for f in os.listdir(category_path)
                     if f.lower().endswith(('.png', '.jpg', '.jpeg'))][:num_samples]

            for j, img_name in enumerate(images):
                img_path = os.path.join(category_path, img_name)
                img = plt.imread(img_path)
                axes[i, j].imshow(img)
                axes[i, j].set_title(f'{category.capitalize()} - {img_name[:15]}...')
                axes[i, j].axis('off')

    plt.suptitle('Sample Images from Dataset', fontsize=16)
    plt.tight_layout()
    plt.show()

display_sample_images(BINARY_TRAIN_PATH)

In [ ]:
# Image parameters
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

# Only rescaling for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    BINARY_TRAIN_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    BINARY_TRAIN_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    BINARY_TEST_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print("Data generators created successfully!")
print(f"Training batches: {len(train_generator)}")
print(f"Validation batches: {len(validation_generator)}")
print(f"Test batches: {len(test_generator)}")
print(f"Class indices: {train_generator.class_indices}")

In [ ]:
def create_model(input_shape=(224, 224, 3)):
    """
    Create a CNN model using transfer learning with EfficientNetB0
    """
    # Load pre-trained EfficientNetB0
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Freeze base model layers initially
    base_model.trainable = False

    # Add custom classification head
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    return model

# Create the model
model = create_model()

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

# Display model summary
model.summary()

In [ ]:
# Define callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        '/content/best_fruit_classifier.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]


In [ ]:
print("Starting model training...")

# First phase: Train with frozen base model
history_phase1 = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

# Second phase: Fine-tuning with unfrozen layers
print("\n" + "="*50)
print("PHASE 2: Fine-tuning with unfrozen layers")
print("="*50)

# Unfreeze the base model
model.layers[0].trainable = True

# Recompile with lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

# Continue training
history_phase2 = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)